In [1]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import textwrap
import json
import os
import ollama

c:\Users\luca-\OneDrive\Desktop\University\#MAGISTRALE\2° ANNO\1° SEMESTRE\Ingegneria dei Dati\hw4\homework4\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def read_prompt_template(prompt_file_path):
    """
    Legge il template del prompt da un file con codifica UTF-8 e gestione degli errori.
    Args:
        prompt_file_path (str): Percorso del file di testo contenente il template del prompt.
    Returns:
        str: Il contenuto del file del template del prompt, o None se si verifica un errore.
    """
    try:
        with open(prompt_file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print(f"Errore: File non trovato al percorso '{prompt_file_path}'")
        return None
    except Exception as e:
        print(f"Errore durante la lettura del file: {e}")
        return None


def interact_with_llm(table_json, model_name, prompt_template,filename):
    """
    Interagisce con un modello linguistico locale (LLM) per generare delle affermazioni.

    Args:
        table_json (dict): Dati JSON tabellari.
        model_name (str): Nome del modello LLM da utilizzare.
        prompt_template (str): Template del prompt.

    Returns:
        dict: Affermazioni generate dall'LLM, o None in caso di errore.
    """
    # Converti il JSON della tabella in una stringa per il prompt
    table_json_str = json.dumps(table_json, indent=4)

    # Crea il prompt completo inserendo la stringa JSON nel template
    full_prompt = prompt_template + "\n" + table_json_str + "\n Richiesta finale: \n Produci unicamente il JSON dei claims estratti, nel formato specificato, senza testo aggiuntivo o spiegazioni."

    # Usa Ollama per inviare il prompt al modello
    try:
        # Costruisci la struttura corretta del messaggio per ollama.chat
        messages = [{"role": "user", "content": full_prompt}]
        response = ollama.chat(model=model_name, messages=messages)
        print("Risposta da LLM:", response)  # Stampa la risposta del modello
    except Exception as e:
        print("Errore durante l'invio del prompt a LLM:", str(e))
        return None

    # Estrai la risposta JSON
    try:
        claims = json.loads(response['message']['content'])
        return claims
    except json.JSONDecodeError as e:
        # Salva la risposta grezza in un file di testo per il debug
        output_directory = '../data/claims'
        output_directory = os.path.abspath(output_directory)
        raw_output_path = os.path.join(output_directory, filename.replace('.json', '_raw_output.txt'))
        
        with open(raw_output_path, 'w', encoding='utf-8') as raw_output_file:
            raw_output_file.write(response['message']['content'])
        print("Impossibile analizzare l'output dell'LLM come JSON:", str(e))
        return None


def process_json_files(input_dir, output_dir, model_name, prompt_file_path):
    """
    Processa più file JSON, estrae le affermazioni usando un modello LLM locale e salva i risultati.

    Args:
        input_dir (str): Directory contenente i file JSON.
        output_dir (str): Directory dove i risultati verranno salvati.
        model_name (str): Nome del modello LLM da usare.
        prompt_file_path (str): Percorso del file di template del prompt.
    """
    # Leggi il template del prompt dal file
    prompt_template = read_prompt_template(prompt_file_path)
    if not prompt_template:  # Se la lettura del file fallisce, esci dalla funzione
        return

    # Assicura che la directory di output esista
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Processa ogni file JSON nella directory di input
    for filename in os.listdir(input_dir):
        if filename.endswith('.json'):
            file_path = os.path.join(input_dir, filename)
            try:
                 with open(file_path, 'r', encoding='utf-8') as file:
                     table_json = json.load(file)
            except FileNotFoundError:
                 print(f"Errore: File non trovato al percorso '{file_path}'")
                 continue  # passa al prossimo file
            except json.JSONDecodeError as e:
                 print(f"Errore: Impossibile analizzare il file JSON '{filename}': {e}")
                 continue  # passa al prossimo file
            except Exception as e:
                 print(f"Errore inatteso durante la lettura del file '{filename}': {e}")
                 continue  # passa al prossimo file

            # Genera le affermazioni usando l'LLM
            claims = interact_with_llm(table_json, model_name, prompt_template, filename)

            if claims is not None:
               # Salva l'output nella directory di output specificata
               output_filename = filename.replace('.json', '_claims.json')
               output_file_path = os.path.join(output_dir, output_filename)
               try:
                    with open(output_file_path, 'w', encoding='utf-8') as output_file:
                         json.dump(claims, output_file, indent=4)
                    print(f"Elaborato {filename} e affermazioni salvate in {output_file_path}")
               except Exception as e:
                     print(f"Errore durante il salvataggio del file '{output_file_path}': {e}")
            else:
                print(f"Errore durante l'estrazione di affermazioni per {filename}. Nessun file salvato.")

In [4]:
input_directory = '../data/json'  # Modifica i percorsi se necessario
output_directory = '../data/claims'


model_name = "llama3.2"
prompt_file_path = 'prompt.txt'  # Il percorso al tuo file di testo del prompt

# Ottieni i percorsi assoluti
absolute_input_directory = os.path.abspath(input_directory)
absolute_output_directory = os.path.abspath(output_directory)

print(f"Percorso assoluto della cartella di input: {absolute_input_directory}")
print(f"Percorso assoluto della cartella di output: {absolute_output_directory}")

# Elabora tutti i file JSON
process_json_files(input_directory, output_directory, model_name, prompt_file_path)

Percorso assoluto della cartella di input: c:\Users\luca-\OneDrive\Desktop\University\#MAGISTRALE\2° ANNO\1° SEMESTRE\Ingegneria dei Dati\hw4\homework4\data\json
Percorso assoluto della cartella di output: c:\Users\luca-\OneDrive\Desktop\University\#MAGISTRALE\2° ANNO\1° SEMESTRE\Ingegneria dei Dati\hw4\homework4\data\claims
Risposta da LLM: model='llama3.2' created_at='2024-12-28T15:06:40.6063975Z' done=True done_reason='stop' total_duration=59848073900 load_duration=22229800 prompt_eval_count=1036 prompt_eval_duration=1532000000 eval_count=1793 eval_duration=58292000000 message=Message(role='assistant', content='[\n    {\n        "specifications": {\n            "0": {"name": "Model", "value": ""},\n            "1": {"name": "# Params.", "value": "1.8B"},\n            "2": {"name": "VS", "value": ""}\n        },\n        "Measure": "VS",\n        "Outcome": "5.3"\n    },\n    {\n        "specifications": {\n            "0": {"name": "Model", "value": ""},\n            "1": {"name": "

In [39]:
def ollama_chat(model_name, prompt):
    """
    Simula una chiamata al modello LLM per ottenere una risposta.
    
    Args:
        model_name (str): Il nome del modello LLM da interrogare.
        prompt (str): Il prompt da inviare al modello.
        
    Returns:
        str: La risposta del modello.
    """
    # Qui si simula una risposta del modello LLM.
    # In un'applicazione reale, qui ci sarebbe il codice per inviare il prompt al modello tramite API.
    if prompt == "Qual è la capitale dell'Italia?":
        return "La capitale dell'Italia è Roma."
    else:
        return "Non sono sicuro della risposta a questa domanda."
# Definisci il nome del modello
model_name = "llama3.2"

    # Crea un prompt semplice
prompt = "Qual è la capitale dell'Italia?"

    # Simula una chiamata al modello LLM
response = ollama_chat(model_name, prompt)

    # Stampa la risposta ottenuta
print("Risposta del modello LLM:", response)

Risposta del modello LLM: La capitale dell'Italia è Roma.
